In [ ]:
import argparse
import redis
from urllib.parse import urlparse
import imageio

In [ ]:
# Set up some vars
initialized_key = 'cats:initialized'

# Set up Redis connection
url = urlparse('redis://127.0.0.1:6379')
conn = redis.Redis(host=url.hostname, port=url.port)
if not conn.ping():
    raise Exception('Redis unavailable')

# Check if this Redis instance had already been initialized
initialized = conn.exists(initialized_key)
if initialized:
    print('Discovered evidence of a previous initialization - skipping.')
    exit(0)

# Load the RedisAI model
print('Loading model - ', end='')
with open('../model.pt', 'rb') as f:
    model = f.read()
    res = conn.execute_command(
        'AI.MODELSET', 
        'model', 
        'TORCH', 
        'CPU', 
        'INPUTS', 'input', 
        'OUTPUTS', 'pred', 
        model)
    print(res)

In [ ]:
# Load the gear
print('Loading gear - ', end='')
with open('gear_stream.py', 'rb') as f:
    gear = f.read()
    res = conn.execute_command('RG.PYEXECUTE', gear)
    print(res)

# Lastly, set a key that indicates initialization has been performed
print('Flag initialization as done - ', end='')

print(conn.set(initialized_key, 'miauw'))